In [1]:
import numpy as np
import time
from functools import reduce

def classical_matrix_mult(A, B):
    n = len(A)
    C = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for k in range(n):
            for j in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

def strassen_matrix_mult(A, B):
    n = len(A)

    if n <= 64:  # Базовый случай: используем классическое умножение
        return classical_matrix_mult(A, B)

    # Разбиваем матрицы на подматрицы
    new_size = n // 2
    A11 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    A12 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    A21 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    A22 = [[0 for _ in range(new_size)] for _ in range(new_size)]

    B11 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    B12 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    B21 = [[0 for _ in range(new_size)] for _ in range(new_size)]
    B22 = [[0 for _ in range(new_size)] for _ in range(new_size)]

    for i in range(new_size):
        for j in range(new_size):
            A11[i][j] = A[i][j]
            A12[i][j] = A[i][j + new_size]
            A21[i][j] = A[i + new_size][j]
            A22[i][j] = A[i + new_size][j + new_size]

            B11[i][j] = B[i][j]
            B12[i][j] = B[i][j + new_size]
            B21[i][j] = B[i + new_size][j]
            B22[i][j] = B[i + new_size][j + new_size]

    # Вычисляем промежуточные матрицы Штрассена
    M1 = strassen_matrix_mult(matrix_add(A11, A22), matrix_add(B11, B22))
    M2 = strassen_matrix_mult(matrix_add(A21, A22), B11)
    M3 = strassen_matrix_mult(A11, matrix_sub(B12, B22))
    M4 = strassen_matrix_mult(A22, matrix_sub(B21, B11))
    M5 = strassen_matrix_mult(matrix_add(A11, A12), B22)
    M6 = strassen_matrix_mult(matrix_sub(A21, A11), matrix_add(B11, B12))
    M7 = strassen_matrix_mult(matrix_sub(A12, A22), matrix_add(B21, B22))

    # Вычисляем подматрицы результата
    C11 = matrix_add(matrix_sub(matrix_add(M1, M4), M5), M7)
    C12 = matrix_add(M3, M5)
    C21 = matrix_add(M2, M4)
    C22 = matrix_add(matrix_sub(matrix_add(M1, M3), M2), M6)

    # Собираем результат
    C = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(new_size):
        for j in range(new_size):
            C[i][j] = C11[i][j]
            C[i][j + new_size] = C12[i][j]
            C[i + new_size][j] = C21[i][j]
            C[i + new_size][j + new_size] = C22[i][j]
    return C

def matrix_add(A, B):
    return [[A[i][j] + B[i][j] for j in range(len(A))] for i in range(len(A))]

def matrix_sub(A, B):
    return [[A[i][j] - B[i][j] for j in range(len(A))] for i in range(len(A))]

def coppersmith_winograd_matrix_mult(A, B):
    # Упрощенная версия (не оптимальная, демонстрационная)
    n = len(A)
    C = [[0 for _ in range(n)] for _ in range(n)]

    # Предварительные вычисления
    beta = [0] * n
    for i in range(n):
        for j in range(n//2):
            beta[i] += A[i][2*j] * A[i][2*j+1]

    gamma = [0] * n
    for j in range(n):
        for i in range(n//2):
            gamma[j] += B[2*i][j] * B[2*i+1][j]

    # Основное вычисление
    for i in range(n):
        for j in range(n):
            C[i][j] = -beta[i] - gamma[j]
            for k in range(n//2):
                C[i][j] += (A[i][2*k] + B[2*k+1][j]) * (A[i][2*k+1] + B[2*k][j])

    return C

def tensor_decomposition_matrix_mult(A, B):
    # Упрощенная демонстрация (на практике используются сложные разложения)
    n = len(A)
    rank = min(n, 10)  # Произвольный выбор ранга

    # Разложение A и B в сумму тензорных произведений
    # Здесь просто имитируем разложение для демонстрации
    C = [[0 for _ in range(n)] for _ in range(n)]
    for r in range(rank):
        # Имитация разложения: берем случайные векторы
        u = [np.random.rand() for _ in range(n)]
        v = [np.random.rand() for _ in range(n)]
        w = [np.random.rand() for _ in range(n)]

        # Добавляем к результату тензорное произведение
        for i in range(n):
            for j in range(n):
                C[i][j] += u[i] * v[j] * sum(w[k] * B[k][j] for k in range(n))
    return C

def generate_matrix(n):
    return np.random.rand(n, n).tolist()

def measure_time(func, A, B, name):
    start = time.time()
    result = func(A, B)
    end = time.time()
    print(f"{name}: {end - start:.4f} секунд")
    return result

if __name__ == "__main__":
    n = 128  # Размер матрицы (должен быть степенью 2 для Штрассена)

    A = generate_matrix(n)
    B = generate_matrix(n)

    # Проверка корректности на маленьких матрицах
    if n <= 8:
        test_A = generate_matrix(2)
        test_B = generate_matrix(2)
        classical = classical_matrix_mult(test_A, test_B)
        strassen = strassen_matrix_mult(test_A, test_B)
        print("Классический:", classical)
        print("Штрассен:", strassen)

    # Измерение времени
    measure_time(classical_matrix_mult, A, B, "Классический алгоритм")
    measure_time(strassen_matrix_mult, A, B, "Алгоритм Штрассена")
    measure_time(coppersmith_winograd_matrix_mult, A, B, "Алгоритм Копперсмита-Винограда")
    measure_time(tensor_decomposition_matrix_mult, A, B, "Алгоритм на тензорных разложениях")

    # Для сравнения: встроенное умножение NumPy (очень оптимизированное)
    np_A = np.array(A)
    np_B = np.array(B)
    start = time.time()
    np_result = np.dot(np_A, np_B)
    end = time.time()
    print(f"NumPy встроенный: {end - start:.4f} секунд")

Классический алгоритм: 0.2122 секунд
Алгоритм Штрассена: 0.2517 секунд
Алгоритм Копперсмита-Винограда: 0.4173 секунд
Алгоритм на тензорных разложениях: 2.6824 секунд
NumPy встроенный: 0.0037 секунд
